In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import adfuller



In [3]:
df=pd.read_csv("data.csv")
df

,City,Date,SO2-H01,SO2-H02,SO2-H03,SO2-H04,SO2-H05,SO2-H06,SO2-H07,SO2-H08,...,NOx-H15,NOx-H16,NOx-H17,NOx-H18,NOx-H19,NOx-H20,NOx-H21,NOx-H22,NOx-H23,NOx-H24
0,Ottawa Downtown,2022-01-01,0.6,0.5,0.5,0.4,0.5,0.5,0.4,0.4,...,7.9,8.4,5.7,6.2,6.0,5.9,4.9,3.7,4.4,3.5
1,Ottawa Downtown,2022-01-02,0.3,0.3,0.3,0.3,0.3,0.3,0.3,0.3,...,3.5,4.0,4.0,3.8,3.6,3.6,3.1,3.8,3.8,3.2
2,Ottawa Downtown,2022-01-03,0.3,0.4,0.4,0.3,0.3,0.3,0.3,0.3,...,9.6,7.3,5.8,13.3,17.9,21.1,7.4,71.9,7.6,24.8
3,Ottawa Downtown,2022-01-04,0.4,0.5,0.4,0.4,0.4,0.6,0.5,0.5,...,15.9,21.2,31.5,35.1,43.3,31.7,21.4,7.4,24.2,27.2
4,Ottawa Downtown,2022-01-05,0.4,0.4,0.4,0.4,0.4,0.4,0.5,0.4,...,18.8,18.5,21.6,27.8,31.9,12.0,12.3,12.4,8.4,9.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2185,Windsor Downtown,2022-12-27,0.8,0.8,0.7,0.7,1.1,0.6,0.7,0.6,...,8.6,9.5,1.3,1.3,1.5,8.0,8.5,9.2,1.4,8.5
2186,Windsor Downtown,2022-12-28,0.1,0.2,0.2,0.3,0.2,0.2,0.1,0.1,...,11.0,1.8,14.2,15.1,9.4,9.4,9.3,1.2,8.1,7.7
2187,Windsor Downtown,2022-12-29,0.5,0.5,0.5,0.5,0.4,0.3,0.2,0.2,...,14.2,12.9,12.2,12.0,11.9,9.5,9.0,9.4,8.1,6.8
2188,Windsor Downtown,2022-12-30,0.2,0.3,0.3,0.2,0.2,0.2,0.2,0.2,...,11.6,9.5,9.4,1.8,15.5,13.0,9.9,13.5,2.3,17.0


# EDA

In [4]:


# Correcting the 'Date' column name and setting it as the index
df['Date '] = pd.to_datetime(df['Date '])
df.set_index('Date ', inplace=True)

# Display Summary Statistics for each pollutant
print(df.describe())

# Time Series Plot for a select few pollutants
# Replace 'SO2-H01', 'SO2-H02', etc., with the pollutants you're interested in
plt.figure(figsize=(12, 6))
plt.plot(df['SO2-H01'], label='SO2-H01')
plt.plot(df['SO2-H02'], label='SO2-H02')
plt.title('Pollutants Over Time')
plt.xlabel('Date')
plt.ylabel('Concentration')
plt.legend()
plt.show()

# Seasonality Analysis - Monthly for a sp ecific pollutant
df['Month'] = df.index.month
monthly_pollutant = df.groupby('Month')['SO2-H01'].mean()
plt.figure(figsize=(10, 5))
sns.barplot(x=monthly_pollutant.index, y=monthly_pollutant.values)
plt.title('Average Monthly Concentration of SO2-H01')
plt.xlabel('Month')
plt.ylabel('Average Concentration')
plt.show()

# Correlation Analysis
plt.figure(figsize=(10, 8))
sns.heatmap(df.corr(), annot=True, cmap='coolwarm')
plt.title('Correlation Matrix')
plt.show()


KeyError: 'Date '

# Calculating Hourly Sub index of each pollutant

In [ ]:

# Define the function to calculate PM2.5 sub-index
def get_PM25_subindex(pi):
    if pi <= 30:
        return pi * 50 / 30
    elif pi <= 60:
        return 50 + (pi - 30) * 50 / 30
    elif pi <= 90:
        return 100 + (pi - 60) * 100 / 30
    elif pi <= 120:
        return 200 + (pi - 90) * 100 / 30
    elif pi <= 250:
        return 300 + (pi - 120) * 100 / 130
    elif pi > 250:
        return 400 + (pi - 250) * 100 / 130
    else:
        return pi

# Apply the function to each hourly PM2.5 column
for hour in range(1, 25):  # For hours 1 through 24
    pm25_col = f'PM2.5-H{hour:02d}'  # PM2.5 column for the hour
    pmi_col = f'PMI-H{hour:02d}'    # New column for the PMI of the hour
    df[pmi_col] = df[pm25_col].apply(get_PM25_subindex)

df


,City,SO2-H01,SO2-H02,SO2-H03,SO2-H04,SO2-H05,SO2-H06,SO2-H07,SO2-H08,SO2-H09,...,PMI-H15,PMI-H16,PMI-H17,PMI-H18,PMI-H19,PMI-H20,PMI-H21,PMI-H22,PMI-H23,PMI-H24
Date,,,,,,,,,,,,,,,,,,,,,
2022-01-01,Ottawa Downtown,0.6,0.5,0.5,0.4,0.5,0.5,0.4,0.4,0.5,...,21.666667,18.333333,10.000000,8.333333,8.333333,8.333333,6.666667,6.666667,6.666667,6.666667
2022-01-02,Ottawa Downtown,0.3,0.3,0.3,0.3,0.3,0.3,0.3,0.3,0.3,...,6.666667,6.666667,8.333333,10.000000,10.000000,10.000000,10.000000,8.333333,8.333333,8.333333
2022-01-03,Ottawa Downtown,0.3,0.4,0.4,0.3,0.3,0.3,0.3,0.3,0.3,...,6.666667,6.666667,10.000000,11.666667,13.333333,15.000000,20.000000,21.666667,1.666667,15.000000
2022-01-04,Ottawa Downtown,0.4,0.5,0.4,0.4,0.4,0.6,0.5,0.5,0.5,...,1.666667,18.333333,18.333333,18.333333,21.666667,21.666667,21.666667,21.666667,21.666667,21.666667
2022-01-05,Ottawa Downtown,0.4,0.4,0.4,0.4,0.4,0.4,0.5,0.4,0.4,...,23.333333,26.666667,3.333333,36.666667,28.333333,18.333333,20.000000,18.333333,20.000000,21.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-27,Windsor Downtown,0.8,0.8,0.7,0.7,1.1,0.6,0.7,0.6,0.4,...,11.666667,1.666667,21.666667,26.666667,28.333333,18.333333,1.666667,18.333333,21.666667,20.000000
2022-12-28,Windsor Downtown,0.1,0.2,0.2,0.3,0.2,0.2,0.1,0.1,0.1,...,15.000000,15.000000,1.666667,20.000000,18.333333,18.333333,1.666667,1.666667,15.000000,13.333333
2022-12-29,Windsor Downtown,0.5,0.5,0.5,0.5,0.4,0.3,0.2,0.2,0.3,...,18.333333,21.666667,25.000000,23.333333,21.666667,20.000000,20.000000,18.333333,15.000000,15.000000


In [ ]:
## O3 Sub-Index calculation
def get_O3_subindex(oi):
    if oi <= 50:
        return oi * 50 / 50
    elif oi <= 100:
        return 50 + (oi - 50) * 50 / 50
    elif oi <= 168:
        return 100 + (oi - 100) * 100 / 68
    elif oi <= 208:
        return 200 + (oi - 168) * 100 /40
    elif oi <= 748:
        return 300 + (oi - 208) * 100 / 539
    elif oi > 748:
        return 400 + (oi - 400) * 100 / 539
    else:
        return oi

# Apply the function to each hourly O3 column
for hour in range(1, 25):
    o3_col = f'O3-H{hour:02d}'
    o3i_col = f'O3I-H{hour:02d}'  # Updated column name for O3 sub-index
    df[o3i_col] = df[o3_col].apply(get_O3_subindex)

df

,City,SO2-H01,SO2-H02,SO2-H03,SO2-H04,SO2-H05,SO2-H06,SO2-H07,SO2-H08,SO2-H09,...,O3I-H15,O3I-H16,O3I-H17,O3I-H18,O3I-H19,O3I-H20,O3I-H21,O3I-H22,O3I-H23,O3I-H24
Date,,,,,,,,,,,,,,,,,,,,,
2022-01-01,Ottawa Downtown,0.6,0.5,0.5,0.4,0.5,0.5,0.4,0.4,0.5,...,13.0,13.0,2.0,23.0,26.0,28.0,3.0,32.0,32.0,3.0
2022-01-02,Ottawa Downtown,0.3,0.3,0.3,0.3,0.3,0.3,0.3,0.3,0.3,...,33.0,33.0,33.0,34.0,33.0,32.0,32.0,31.0,31.0,31.0
2022-01-03,Ottawa Downtown,0.3,0.4,0.4,0.3,0.3,0.3,0.3,0.3,0.3,...,32.0,31.0,27.0,26.0,21.0,18.0,5.0,4.0,12.0,16.0
2022-01-04,Ottawa Downtown,0.4,0.5,0.4,0.4,0.4,0.6,0.5,0.5,0.5,...,24.0,19.0,1.0,7.0,3.0,7.0,12.0,13.0,1.0,1.0
2022-01-05,Ottawa Downtown,0.4,0.4,0.4,0.4,0.4,0.4,0.5,0.4,0.4,...,21.0,19.0,16.0,11.0,12.0,26.0,24.0,24.0,25.0,19.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-27,Windsor Downtown,0.8,0.8,0.7,0.7,1.1,0.6,0.7,0.6,0.4,...,31.0,28.0,25.0,23.0,22.0,24.0,24.0,22.0,21.0,24.0
2022-12-28,Windsor Downtown,0.1,0.2,0.2,0.3,0.2,0.2,0.1,0.1,0.1,...,3.0,3.0,25.0,24.0,32.0,35.0,35.0,32.0,34.0,33.0
2022-12-29,Windsor Downtown,0.5,0.5,0.5,0.5,0.4,0.3,0.2,0.2,0.3,...,25.0,26.0,27.0,25.0,24.0,24.0,24.0,25.0,26.0,25.0


In [ ]:
def get_NO2_subindex(ni):
    if ni <= 40:
        return ni * 50 / 40
    elif ni <= 80:
        return 50 + (ni - 40) * 50 / 40
    elif ni <= 180:
        return 100 + (ni - 80) * 100 / 100
    elif ni <= 280:
        return 200 + (ni - 180) * 100 / 100
    elif ni <= 400:
        return 300 + (ni - 280) * 100 / 120
    elif ni > 400:
        return 400 + (ni - 400) * 100 / 120
    else:
        return ni# Apply the function to each hourly NO2 column

for hour in range(1, 25):
    no2_col = f'NO2-H{hour:02d}'
    ni_col = f'NI-H{hour:02d}'  # Updated column name for NO2 sub-index
    df[ni_col] = df[no2_col].apply(get_NO2_subindex)
df

,City,SO2-H01,SO2-H02,SO2-H03,SO2-H04,SO2-H05,SO2-H06,SO2-H07,SO2-H08,SO2-H09,...,NI-H15,NI-H16,NI-H17,NI-H18,NI-H19,NI-H20,NI-H21,NI-H22,NI-H23,NI-H24
Date,,,,,,,,,,,,,,,,,,,,,
2022-01-01,Ottawa Downtown,0.6,0.5,0.5,0.4,0.5,0.5,0.4,0.4,0.5,...,7.625,7.875,5.750,5.875,5.500,5.250,4.750,3.625,4.000,3.500
2022-01-02,Ottawa Downtown,0.3,0.3,0.3,0.3,0.3,0.3,0.3,0.3,0.3,...,3.125,3.750,3.750,3.625,3.500,3.500,3.000,3.625,3.625,3.125
2022-01-03,Ottawa Downtown,0.3,0.4,0.4,0.3,0.3,0.3,0.3,0.3,0.3,...,8.625,7.625,1.250,15.750,20.750,2.500,44.625,8.625,33.875,27.875
2022-01-04,Ottawa Downtown,0.4,0.5,0.4,0.4,0.4,0.6,0.5,0.5,0.5,...,17.000,22.875,31.750,34.625,39.500,33.625,24.250,23.125,27.250,27.875
2022-01-05,Ottawa Downtown,0.4,0.4,0.4,0.4,0.4,0.4,0.5,0.4,0.4,...,18.250,20.000,24.250,30.625,31.000,7.750,13.875,13.875,9.375,10.125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-27,Windsor Downtown,0.8,0.8,0.7,0.7,1.1,0.6,0.7,0.6,0.4,...,8.125,9.750,11.125,11.375,11.625,8.750,9.500,10.375,11.500,9.375
2022-12-28,Windsor Downtown,0.1,0.2,0.2,0.3,0.2,0.2,0.1,0.1,0.1,...,10.875,11.125,15.875,17.250,10.875,10.750,10.625,11.875,9.375,8.875
2022-12-29,Windsor Downtown,0.5,0.5,0.5,0.5,0.4,0.3,0.2,0.2,0.3,...,15.000,14.625,14.000,13.875,13.875,10.875,10.375,10.625,9.375,7.875


In [ ]:
def get_SO2_subindex(si):
    if si <= 40:
        return si * 50 / 40
    elif si <= 80:
        return 50 + (si - 40) * 50 / 40
    elif si <= 380:
        return 100 + (si - 80) * 100 / 300
    elif si <= 800:
        return 200 + (si - 380) * 100 / 420
    elif si <= 1600:
        return 300 + (si - 800) * 100 / 800
    elif si > 1600:
        return 400 + (si - 1600) * 100 / 800
    else:
        return si
    # Apply the function to each hourly SO2 column
for hour in range(1, 25):
    so2_col = f'SO2-H{hour:02d}'
    si_col = f'SI-H{hour:02d}'  # Updated column name for SO2 sub-index
    df[si_col] = df[so2_col].apply(get_SO2_subindex)
df

,City,SO2-H01,SO2-H02,SO2-H03,SO2-H04,SO2-H05,SO2-H06,SO2-H07,SO2-H08,SO2-H09,...,SI-H15,SI-H16,SI-H17,SI-H18,SI-H19,SI-H20,SI-H21,SI-H22,SI-H23,SI-H24
Date,,,,,,,,,,,,,,,,,,,,,
2022-01-01,Ottawa Downtown,0.6,0.5,0.5,0.4,0.5,0.5,0.4,0.4,0.5,...,0.500,0.500,0.500,0.500,0.500,0.375,0.375,0.375,0.375,0.375
2022-01-02,Ottawa Downtown,0.3,0.3,0.3,0.3,0.3,0.3,0.3,0.3,0.3,...,0.375,0.375,0.375,0.375,0.375,0.375,0.375,0.375,0.375,0.375
2022-01-03,Ottawa Downtown,0.3,0.4,0.4,0.3,0.3,0.3,0.3,0.3,0.3,...,0.500,0.500,0.500,0.500,0.625,0.500,0.625,0.750,0.625,0.500
2022-01-04,Ottawa Downtown,0.4,0.5,0.4,0.4,0.4,0.6,0.5,0.5,0.5,...,0.625,0.625,0.625,0.625,0.625,0.625,0.500,0.500,0.500,0.625
2022-01-05,Ottawa Downtown,0.4,0.4,0.4,0.4,0.4,0.4,0.5,0.4,0.4,...,0.875,0.875,1.125,1.000,0.750,0.500,0.500,0.500,0.500,0.500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-27,Windsor Downtown,0.8,0.8,0.7,0.7,1.1,0.6,0.7,0.6,0.4,...,0.625,0.375,0.250,0.125,0.125,0.125,0.125,0.125,0.125,0.125
2022-12-28,Windsor Downtown,0.1,0.2,0.2,0.3,0.2,0.2,0.1,0.1,0.1,...,1.125,1.000,0.500,0.250,0.250,0.500,0.500,0.500,0.625,0.625
2022-12-29,Windsor Downtown,0.5,0.5,0.5,0.5,0.4,0.3,0.2,0.2,0.3,...,0.500,0.500,0.500,0.375,0.375,0.250,0.250,0.250,0.250,0.250


In [ ]:
# Apply the function to each hourly CO column
def get_CO_subindex(ci):
    if ci <= 1:
        return ci * 50 / 1
    elif ci <= 2:
        return 50 + (ci - 1) * 50 / 1
    elif ci <= 10:
        return 100 + (ci - 2) * 100 / 8
    elif ci <= 17:
        return 200 + (ci - 10) * 100 / 7
    elif ci <= 34:
        return 300 + (ci - 17) * 100 / 17
    elif ci > 34:
        return 400 + (ci - 34) * 100 / 17
    else:
        return 0
for hour in range(1, 25):
    co_col = f'CO-H{hour:02d}'
    ci_col = f'CI-H{hour:02d}'  # Updated column name for CO sub-index
    df[ci_col] = df[co_col].apply(get_CO_subindex)
df

C:\Users\500223~2\AppData\Local\Temp/ipykernel_1840/2116071872.py:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[ci_col] = df[co_col].apply(get_CO_subindex)


,City,SO2-H01,SO2-H02,SO2-H03,SO2-H04,SO2-H05,SO2-H06,SO2-H07,SO2-H08,SO2-H09,...,CI-H15,CI-H16,CI-H17,CI-H18,CI-H19,CI-H20,CI-H21,CI-H22,CI-H23,CI-H24
Date,,,,,,,,,,,,,,,,,,,,,
2022-01-01,Ottawa Downtown,0.6,0.5,0.5,0.4,0.5,0.5,0.4,0.4,0.5,...,9.5,10.0,10.0,10.0,10.0,10.5,10.5,10.5,11.0,10.5
2022-01-02,Ottawa Downtown,0.3,0.3,0.3,0.3,0.3,0.3,0.3,0.3,0.3,...,9.5,10.0,10.0,10.0,10.0,10.5,10.5,10.5,11.0,10.5
2022-01-03,Ottawa Downtown,0.3,0.4,0.4,0.3,0.3,0.3,0.3,0.3,0.3,...,9.5,10.0,10.0,10.0,10.0,10.5,10.5,10.5,11.0,10.5
2022-01-04,Ottawa Downtown,0.4,0.5,0.4,0.4,0.4,0.6,0.5,0.5,0.5,...,9.5,10.0,10.0,10.0,10.0,10.5,10.5,10.5,11.0,10.5
2022-01-05,Ottawa Downtown,0.4,0.4,0.4,0.4,0.4,0.4,0.5,0.4,0.4,...,9.5,10.0,10.0,10.0,10.0,10.5,10.5,10.5,11.0,10.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-27,Windsor Downtown,0.8,0.8,0.7,0.7,1.1,0.6,0.7,0.6,0.4,...,11.5,12.5,12.0,12.5,12.5,11.5,11.5,11.5,8.0,9.0
2022-12-28,Windsor Downtown,0.1,0.2,0.2,0.3,0.2,0.2,0.1,0.1,0.1,...,10.0,8.5,8.5,6.5,5.0,45.0,5.0,6.5,10.5,10.5
2022-12-29,Windsor Downtown,0.5,0.5,0.5,0.5,0.4,0.3,0.2,0.2,0.3,...,13.5,12.5,12.5,12.0,13.0,12.5,12.0,12.0,10.0,8.5


# Hourly AQI Generattion

In [ ]:
# Function to calculate AQI from the sub-indices
def calculate_hourly_aqi(row):
    # Extracting the sub-indices for each pollutant for a specific hour
    hourly_aqi_values = [row[f'CI-H{hour:02d}'], row[f'SI-H{hour:02d}'], 
                         row[f'NI-H{hour:02d}'], row[f'O3I-H{hour:02d}'], 
                         row[f'PMI-H{hour:02d}']]
    # The AQI is the maximum of these sub-indices
    return max(hourly_aqi_values)

# Calculate AQI for each hour and store in new columns
for hour in range(1, 25):
    aqi_col = f'AQI-H{hour:02d}'
    df[aqi_col] = df.apply(calculate_hourly_aqi, axis=1)

# Now 'df' contains the AQI for each hour
df

,City,SO2-H01,SO2-H02,SO2-H03,SO2-H04,SO2-H05,SO2-H06,SO2-H07,SO2-H08,SO2-H09,...,AQI-H15,AQI-H16,AQI-H17,AQI-H18,AQI-H19,AQI-H20,AQI-H21,AQI-H22,AQI-H23,AQI-H24
Date,,,,,,,,,,,,,,,,,,,,,
2022-01-01,Ottawa Downtown,0.6,0.5,0.5,0.4,0.5,0.5,0.4,0.4,0.5,...,21.666667,18.333333,10.00,23.000000,26.000000,28.000000,10.500000,32.000000,32.000000,10.500000
2022-01-02,Ottawa Downtown,0.3,0.3,0.3,0.3,0.3,0.3,0.3,0.3,0.3,...,33.000000,33.000000,33.00,34.000000,33.000000,32.000000,32.000000,31.000000,31.000000,31.000000
2022-01-03,Ottawa Downtown,0.3,0.4,0.4,0.3,0.3,0.3,0.3,0.3,0.3,...,32.000000,31.000000,27.00,26.000000,21.000000,18.000000,44.625000,21.666667,33.875000,27.875000
2022-01-04,Ottawa Downtown,0.4,0.5,0.4,0.4,0.4,0.6,0.5,0.5,0.5,...,24.000000,22.875000,31.75,34.625000,39.500000,33.625000,24.250000,23.125000,27.250000,27.875000
2022-01-05,Ottawa Downtown,0.4,0.4,0.4,0.4,0.4,0.4,0.5,0.4,0.4,...,23.333333,26.666667,24.25,36.666667,31.000000,26.000000,24.000000,24.000000,25.000000,21.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-27,Windsor Downtown,0.8,0.8,0.7,0.7,1.1,0.6,0.7,0.6,0.4,...,31.000000,28.000000,25.00,26.666667,28.333333,24.000000,24.000000,22.000000,21.666667,24.000000
2022-12-28,Windsor Downtown,0.1,0.2,0.2,0.3,0.2,0.2,0.1,0.1,0.1,...,15.000000,15.000000,25.00,24.000000,32.000000,45.000000,35.000000,32.000000,34.000000,33.000000
2022-12-29,Windsor Downtown,0.5,0.5,0.5,0.5,0.4,0.3,0.2,0.2,0.3,...,25.000000,26.000000,27.00,25.000000,24.000000,24.000000,24.000000,25.000000,26.000000,25.000000


# Overall AQI

In [ ]:
# Calculate the average of the hourly AQI values
hourly_aqi_columns = [f'AQI-H{hour:02d}' for hour in range(1, 25)]
df['Overall_AQI_Avg'] = df[hourly_aqi_columns].mean(axis=1)
df


,City,SO2-H01,SO2-H02,SO2-H03,SO2-H04,SO2-H05,SO2-H06,SO2-H07,SO2-H08,SO2-H09,...,AQI-H16,AQI-H17,AQI-H18,AQI-H19,AQI-H20,AQI-H21,AQI-H22,AQI-H23,AQI-H24,Overall_AQI_Avg
Date,,,,,,,,,,,,,,,,,,,,,
2022-01-01,Ottawa Downtown,0.6,0.5,0.5,0.4,0.5,0.5,0.4,0.4,0.5,...,18.333333,10.00,23.000000,26.000000,28.000000,10.500000,32.000000,32.000000,10.500000,34.071181
2022-01-02,Ottawa Downtown,0.3,0.3,0.3,0.3,0.3,0.3,0.3,0.3,0.3,...,33.000000,33.00,34.000000,33.000000,32.000000,32.000000,31.000000,31.000000,31.000000,28.312500
2022-01-03,Ottawa Downtown,0.3,0.4,0.4,0.3,0.3,0.3,0.3,0.3,0.3,...,31.000000,27.00,26.000000,21.000000,18.000000,44.625000,21.666667,33.875000,27.875000,27.001736
2022-01-04,Ottawa Downtown,0.4,0.5,0.4,0.4,0.4,0.6,0.5,0.5,0.5,...,22.875000,31.75,34.625000,39.500000,33.625000,24.250000,23.125000,27.250000,27.875000,25.494792
2022-01-05,Ottawa Downtown,0.4,0.4,0.4,0.4,0.4,0.4,0.5,0.4,0.4,...,26.666667,24.25,36.666667,31.000000,26.000000,24.000000,24.000000,25.000000,21.666667,24.791667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-27,Windsor Downtown,0.8,0.8,0.7,0.7,1.1,0.6,0.7,0.6,0.4,...,28.000000,25.00,26.666667,28.333333,24.000000,24.000000,22.000000,21.666667,24.000000,23.788194
2022-12-28,Windsor Downtown,0.1,0.2,0.2,0.3,0.2,0.2,0.1,0.1,0.1,...,15.000000,25.00,24.000000,32.000000,45.000000,35.000000,32.000000,34.000000,33.000000,27.166667
2022-12-29,Windsor Downtown,0.5,0.5,0.5,0.5,0.4,0.3,0.2,0.2,0.3,...,26.000000,27.00,25.000000,24.000000,24.000000,24.000000,25.000000,26.000000,25.000000,25.263889


In [ ]:
def ad_fuller(timeseries):
    print('Dickey-Fuller Test indicates:')
    df_test = adfuller(timeseries, regression='ct', autolag='AIC')
    output = pd.Series(df_test[0:4], index=['Test Statistic', 'p-value', '#Lags Used', 'Number of Observations Used'])
    print(output)

# Function for SARIMA Forecast with Accuracy Metrics
def sarima_forecast_with_accuracy(df, Overall_AQI_Avg, start_date='2022-01-01', test_size=0.4, forecast_period=365):
    # Convert the index to datetime
    df.index = pd.to_datetime(df.index)

    # Split the data into training and testing sets
    train_size = int(len(df) * (1 - test_size))
    train, test = df.iloc[:train_size], df.iloc[train_size:]

    # Iterate through each target column for forecasting
    for Overall_AQI_Avg in Overall_AQI_Avg:
        print(f"\nForecasting for {Overall_AQI_Avg}")

        # Benchmark with Linear Regression
        regression = LinearRegression().fit(train.index.month.values.reshape(-1, 1), train[Overall_AQI_Avg])
        linear_pred = regression.predict(test.index.month.values.reshape(-1, 1))
        print("Linear Regression Mean squared error: %.3f" % mean_squared_error(test[Overall_AQI_Avg], linear_pred))
        print("Linear Regression Coefficient of determination: %.3f" % r2_score(test[Overall_AQI_Avg], linear_pred))

        # SARIMA model for the target column
        model = SARIMAX(train[Overall_AQI_Avg], order=(1, 1, 1), seasonal_order=(1, 1, 1, 12),
                        enforce_stationarity=False, enforce_invertibility=False)
        results = model.fit(disp=False)

        # Forecast future values
        forecast_index = pd.date_range(start=test.index.min(), periods=forecast_period, freq='D')
        forecast = results.get_forecast(steps=forecast_period, index=forecast_index)

        # Evaluate model performance on the test set
        predictions = results.get_prediction(start=test.index.min(), end=test.index.max(), dynamic=False)
        mse = ((predictions.predicted_mean - test[Overall_AQI_Avg]) ** 2).mean()

        # Print Mean Squared Error (MSE) as a measure of model performance
        print(f'SARIMA Mean Squared Error (MSE) for {Overall_AQI_Avg}: {mse:.2f}')

        # R-squared (coefficient of determination) as a measure of explained variance
        r2 = r2_score(test[Overall_AQI_Avg], predictions.predicted_mean)
        print(f'R-squared for {Overall_AQI_Avg}: {r2:.3f}')

        # Plot original data, testing data, and forecasted values
        plt.figure(figsize=(12, 6))
        plt.plot(train.index, train[Overall_AQI_Avg], label='Training Data')
        plt.plot(test.index, test[Overall_AQI_Avg], label='Testing Data', color='blue')
        plt.plot(forecast_index, forecast.predicted_mean, label=f'SARIMA Forecasted Values ({Overall_AQI_Avg})', color='red')
        plt.plot(test.index, linear_pred, label=f'Linear Regression Benchmark ({Overall_AQI_Avg})', color='green')
        plt.title(f'SARIMA Forecast vs. Linear Regression Benchmark ({Overall_AQI_Avg})')
        plt.xlabel('Date')
        plt.ylabel(Overall_AQI_Avg)
        plt.legend()
        plt.show()

        # Residual Analysis
        residuals = test[Overall_AQI_Avg] - predictions.predicted_mean
        residuals.plot(label='Residuals', color='green')
        plt.axhline(0, color='black', linestyle='--', linewidth=1, label='Zero Line (Mean Residuals)')
        plt.title(f'Residual Analysis ({Overall_AQI_Avg})')
        plt.xlabel('Date')
        plt.ylabel('Residuals')
        plt.legend()
        plt.show()

# Apply the SARIMA forecast function with accuracy metrics
sarima_forecast_with_accuracy(df, Overall_AQI_Avg)

NameError: name 'Overall_AQI_Avg' is not defined